# **5. Объединение DataFrame: знакомимся с новыми данными**

✍ На практике источники данных редко ограничиваются одной таблицей. Например, если мы работаем с базой данных, то данные в ней могут быть представлены в виде нескольких десятков таблиц, каждая из которых несёт отдельную информацию. Если вы делаете выгрузку из базы напрямую, не объединяя таблицы в единую структуру средствами SQL, вам необходимо знать, как работать с такими таблицами средствами Pandas. 

С КАКИМИ ДАННЫМИ МЫ РАБОТАЕМ?

В этой части модуля мы будем работать с популярным датасетом MovieLens, в котором собраны логи некоторой рекомендательной системы фильмов.

Скачать таблицы, с которыми мы будем работать, можно здесь (файлы находятся в zip-архиве — распакуйте архив, прежде чем продолжать работу!)

Наши данные представляют собой четыре таблицы:

ratings1 и ratings2 — таблицы с данными о выставленных пользователями оценках фильмов. Они имеют одинаковую структуру и типы данных — на самом деле это две части одной таблицы с оценками фильмов.

![Py12_5_ratings1_ratings2.png](attachment:Py12_5_ratings1_ratings2.png)

userId — уникальный идентификатор пользователя, который выставил оценку;

movieId — уникальный идентификатор фильма;

rating — рейтинг фильма.

dates — таблица с датами выставления всех оценок.


date — дата и время выставления оценки фильму.

![Py12_5_dates.png](attachment:Py12_5_dates.png)

movies — таблица с информацией о фильмах.

![Py12_5_movies.png](attachment:Py12_5_movies.png)

movieId — уникальный идентификатор фильма;
title — название фильма и год его выхода;
genres — жанры фильма.
Итак, представим, что нам надо получить единую таблицу, в которой будут собраны рейтинги, даты выставления рейтингов, а также информация о фильмах. Вот как мы будем действовать:

1
Склеим таблицы ratings1 и ratings2 в единую структуру.

Термин «склеить» в данном случае обозначает конкатенацию — присоединение одной таблицы к другой.
2
К полученной таблице с рейтингами подсоединим столбец с датой проставления рейтинга, склеив столбцы таблиц между собой.

3
Присоединим к нашей таблице информацию о названиях и жанрах фильмов.

ЗАЧЕМ ХРАНИТЬ ДАННЫЕ В РАЗНЫХ ТАБЛИЦАХ?

Конечно, здорово, если все необходимые данные лежат в одной таблице, но на практике такое случается редко по двум объективным причинам:

Часто данные формируются несколькими независимыми процессами, каждый из которых хранит данные в своей таблице.

Например, данные для отчёта по продажам могут состоять из списка банковских транзакций, курсов валют от Центробанка и планов отдела продаж из внутренней CRM. Все эти три таблицы, скорее всего, будут формироваться независимыми друг от друга системами. Объединять их в один отчёт придётся вам.

Хранить все данные в одной таблице часто очень накладно для ёмкости диска.

Например, названия фильмов в наших данных хранятся в отдельной небольшой таблице. А в логах, которые могут растягиваться на многие миллионы строк, вместо названия фильма стоит его идентификатор. Числовой идентификатор фильма занимает на диске гораздо меньше места, чем длинное название, поэтому логи с идентификаторами будут занимать гораздо меньше места, чем единая таблица с названиями.

✍ Прежде чем приступать к объединению таблиц, предлагаем вам исследовать информацию, которая в них содержится ↓

![5.1.PNG](attachment:5.1.PNG)

In [1]:
import numpy as np

In [2]:
import pandas as pd

*Задание 5.2*

Сколько уникальных фильмов представлено в таблице movies?

(9742)

In [3]:
movies_data = pd.read_csv('data/movies.csv')
movies_df = movies_data.copy()
movies_df.nunique ()

movieId    9742
title      9737
genres      951
dtype: int64

*Задание 5.3*

Сколько уникальных пользователей в таблице ratings1?

(274)

In [6]:
ratings1_data = pd.read_csv('data/ratings1.csv')
ratings1_df = ratings1_data.copy()
ratings1_df.nunique ()

userId      274
movieId    6219
rating       10
dtype: int64

*Задание 5.4*

В каком году было выставлено больше всего оценок?
Для ответа на этот вопрос используйте таблицу dates.

(2000)

In [24]:
dates = pd.read_csv('data/dates.csv') 
dates_df = dates.copy()
dates_df.head()
dates_df['date'] = pd.to_datetime(dates_df['date'], dayfirst=True)
years_sr = dates_df['date'].dt.year
years_sr.value_counts()

C:\Users\Karina\AppData\Local\Temp\ipykernel_12008\1660309524.py:4: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  dates_df['date'] = pd.to_datetime(dates_df['date'], dayfirst=True)


date
2000    10061
2017     8198
2007     7114
2016     6703
2015     6616
2018     6418
1996     6040
2005     5813
2012     4656
2008     4351
2009     4158
2006     4059
2003     4014
2001     3922
2002     3478
2004     3279
1999     2439
2010     2301
1997     1916
2011     1690
2013     1664
2014     1439
1998      507
Name: count, dtype: int64

*********
Все просто. Только стоит понять что dates_df - это датафрейм,  а years_df - это не датафрейм, а объект  типа series. И к ним суффикс - sr. Тогда путаницы будет меньше. В чем разница - следует разобраться и обновить 
знания.

Таблица имеет строки и столбцы с наименованиями. Серия по Панде не имеет ни столбцов, ни строк. У серии есть только два свойства: index- левый столбец, доступен в виде списка и  values - правый столбец и тоже доступен в виде списка. Точнее в виде array. Это нужно будет знать при построении различных диаграмм

Но серию мы можем преобразовать в датафрейм. Это будет в разведывательном анализе данных.